In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# installing necessary applications for 
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/"
!sudo apt-get update
!sudo apt-get -y install cuda

# Install JoeyNMT
! git clone https://github.com/may-/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.9.0
! pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

--2021-09-10 21:02:38--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2021-09-10 21:02:38 (16.8 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.in1HVQiOxR/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Er

In [ ]:
import os
from os import path
tag = "pmi_nhs_baseline"
tokenized_data = "/content/drive/MyDrive/DA/updated_data_tok_files"
src_lang = "en"
tgt_lang = "hi"
bpe_operations = 5000
bpe_file = "len_full.bpe.codes." + str(bpe_operations)
bpe_folder = src_lang + "_" + tgt_lang + "_" + tag + "_" + str(bpe_operations)
bpe_data = "/content/drive/MyDrive/DA/" + bpe_folder + "/"

os.environ["src_lang"] = src_lang
os.environ["tgt_lang"] = tgt_lang
os.environ["tokenized_data"] = tokenized_data
os.environ["bpe_file"] = bpe_file
os.environ["bpe_operations"] = str(bpe_operations)
os.environ["jnmt_data_path"] = path.join("joeynmt", "data", src_lang + tgt_lang)

! mkdir -p $jnmt_data_path
! mkdir -p $bpe_data

In [ ]:
### Learn BPEs on training data
from os import path

os.environ["jnmt_data_path"] = path.join("joeynmt", "data", src_lang + tgt_lang)  
! subword-nmt learn-joint-bpe-and-vocab --input $tokenized_data/train_tok.$src_lang $tokenized_data/train_tok.$tgt_lang -s $bpe_operations -o $bpe_file --write-vocabulary vocab.$src_lang vocab.$tgt_lang

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c $bpe_file --vocabulary vocab.$src_lang < $tokenized_data/train_tok.$src_lang > train.bpe.$src_lang
! subword-nmt apply-bpe -c $bpe_file --vocabulary vocab.$tgt_lang < $tokenized_data/train_tok.$tgt_lang > train.bpe.$tgt_lang

! subword-nmt apply-bpe -c $bpe_file --vocabulary vocab.$src_lang < $tokenized_data/dev_tok.$src_lang > dev.bpe.$src_lang
! subword-nmt apply-bpe -c $bpe_file --vocabulary vocab.$tgt_lang < $tokenized_data/dev_tok.$tgt_lang > dev.bpe.$tgt_lang


! subword-nmt apply-bpe -c $bpe_file --vocabulary vocab.$src_lang < $tokenized_data/test_tok.$src_lang > test.bpe.$src_lang
! subword-nmt apply-bpe -c $bpe_file --vocabulary vocab.$tgt_lang < $tokenized_data/test_tok.$tgt_lang > test.bpe.$tgt_lang


In [ ]:
!cp *.bpe.* $jnmt_data_path

#!cp *.bpe.* $bpe_data

In [ ]:
!mkdir -p $bpe_data/model_data
!cp /content/joeynmt/data/enhi/* $bpe_data/model_data
!cp *vocab* $bpe_data/model_data

In [ ]:
# # Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py $jnmt_data_path/train.bpe.$src_lang $jnmt_data_path/train.bpe.$tgt_lang --output_path $jnmt_data_path/vocab.txt

In [ ]:
model_type = "bpe"
use_cuda = True
gdrive_path = "content/drive/MyDrive/domain_adaptation/experiments/%s-%s-%s" % (src_lang, tgt_lang, tag)

os.environ["gdrive_path"] = gdrive_path

# Create the Joey NMT config file

config = """
name: "{name}_transformer"

data:
    src: "{src_lang}"
    trg: "{tgt_lang}"
    train: "data/{name}/train.{model_type}"
    dev:   "data/{name}/dev.{model_type}"
    test:   "data/{name}/test.{model_type}"
    level: "{model_type}"
    lowercase: False
    max_sent_length: 60
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer_{tag}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: {use_cuda}
    max_output_length: 130
    print_valid_sents: [0, 1, 2, 3]
    keep_best_ckpts: 3
    
model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name = src_lang+tgt_lang, gdrive_path = os.environ["gdrive_path"], src_lang = src_lang, tgt_lang = tgt_lang, model_type = model_type, use_cuda = use_cuda, tag = tag)
with open("joeynmt/configs/transformer_{name}.yaml".format(name = src_lang+tgt_lang), 'w') as f:
    f.write(config)

In [ ]:
! pip install --upgrade sacrebleu==2.0.0

In [ ]:
# Train the model
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src_lang$tgt_lang.yaml

2021-09-10 16:23:47,854 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-10 16:23:47,920 - INFO - joeynmt.data - Loading training data...
2021-09-10 16:23:48,918 - INFO - joeynmt.data - Building vocabulary...
2021-09-10 16:23:49,344 - INFO - joeynmt.data - Loading dev data...
2021-09-10 16:23:49,353 - INFO - joeynmt.data - Loading test data...
2021-09-10 16:23:49,358 - INFO - joeynmt.data - Data loaded.
2021-09-10 16:23:49,358 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-10 16:23:49,570 - INFO - joeynmt.model - Enc-dec model built.
2021-09-10 16:23:51,695 - INFO - joeynmt.training - Total params: 12422400
2021-09-10 16:23:57,490 - INFO - joeynmt.helpers - cfg.name                           : enhi_transformer
2021-09-10 16:23:57,491 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021-09-10 16:23:57,491 - INFO - joeynmt.helpers - cfg.data.trg                       : hi
2021-09-10 16:23:57,491 - INFO - joeynmt.helpers - cfg.data.t

In [ ]:
copy_model_to = "/content/drive/MyDrive/DA/model_" + tag + "_100epochs_new"
!mkdir -p $copy_model_to
os.environ["copy_model_to"] = copy_model_to

!cp -R /content/joeynmt/models/enhi_transformer_pmi_nhs_baseline/ $copy_model_to

In [ ]:
# Output our validation accuracy
! cat "/content/joeynmt/models/enhi_transformer_pmi_nhs_baseline/validations.txt"

Steps: 1000	Loss: 61840.17188	PPL: 60.75191	bleu: 2.05363	LR: 0.00030000	*
Steps: 2000	Loss: 53459.46875	PPL: 34.82158	bleu: 3.91663	LR: 0.00030000	*
Steps: 3000	Loss: 48293.35938	PPL: 24.70872	bleu: 5.71012	LR: 0.00030000	*
Steps: 4000	Loss: 44916.17969	PPL: 19.74456	bleu: 9.27238	LR: 0.00030000	*
Steps: 5000	Loss: 42081.63281	PPL: 16.35668	bleu: 10.14441	LR: 0.00030000	*
Steps: 6000	Loss: 40126.08984	PPL: 14.36463	bleu: 12.60291	LR: 0.00030000	*
Steps: 7000	Loss: 38405.04297	PPL: 12.81318	bleu: 13.77674	LR: 0.00030000	*
Steps: 8000	Loss: 37149.04297	PPL: 11.78778	bleu: 15.72725	LR: 0.00030000	*
Steps: 9000	Loss: 35885.71484	PPL: 10.83917	bleu: 16.58240	LR: 0.00030000	*
Steps: 10000	Loss: 34824.90234	PPL: 10.10184	bleu: 17.58666	LR: 0.00030000	*
Steps: 11000	Loss: 33942.92188	PPL: 9.52715	bleu: 18.82089	LR: 0.00030000	*
Steps: 12000	Loss: 32986.49219	PPL: 8.94084	bleu: 18.85669	LR: 0.00030000	*
Steps: 13000	Loss: 32245.64453	PPL: 8.51160	bleu: 18.98343	LR: 0.00030000	*
Steps: 14000	Lo

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "/content/joeynmt/models/enhi_transformer_pmi_nhs_baseline/config.yaml"

2021-09-10 20:01:03,711 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-10 20:01:03,711 - INFO - joeynmt.data - Building vocabulary...
2021-09-10 20:01:04,179 - INFO - joeynmt.data - Loading dev data...
2021-09-10 20:01:04,185 - INFO - joeynmt.data - Loading test data...
2021-09-10 20:01:04,191 - INFO - joeynmt.data - Data loaded.
2021-09-10 20:01:04,221 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2021-09-10 20:01:04,221 - INFO - joeynmt.prediction - Loading model from models/enhi_transformer_pmi_nhs_baseline/latest.ckpt
2021-09-10 20:01:08,317 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-10 20:01:08,513 - INFO - joeynmt.model - Enc-dec model built.
2021-09-10 20:01:08,584 - INFO - joeynmt.prediction - Decoding on dev set (data/enhi/dev.bpe.hi)...
2021-09-10 20:01:22,307 - INFO - joeynmt.prediction -  dev bleu[13a]:  29.72 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-09-10 20:01:22,3